In [ ]:
import numpy as np
import quadrature4
import curve
import matplotlib.pyplot as plt
import json

In [ ]:
def figure8_generate(resolution=30, twist=0.2):
    points = []
    for i in range(resolution):
        theta = 2 * np.pi / resolution * i
        points.append(np.array([np.cos(theta), np.sin(2*theta), twist * np.sin(theta)]))
    figure8 = curve.Curve(points)
    return figure8

In [ ]:
deltaT=0.004
Lambda=0.01
M=1000
alpha=2

In [ ]:
# L2 Gradient Flow
figure8 = figure8_generate()

deltaT=0.004
Lambda=0.01
M=1000
curve = figure8
L2Energies = []

for t in range(M):
    #print(f"{t} / {M} ({t/M*100:.2f}%)")
    curve = curve - (quadrature4.dEnergy(curve, 2, 4)) * deltaT
    L2Energies.append(quadrature4.energy(curve.list_of_points))
    print(f"Progress: {t} / {M}")

In [ ]:
# Save Progress
with open("L2Energies.json", "w") as file:
    json.dump(L2Energies, file)
    file.close()

In [ ]:
#H1 Gradient Flow

# Input curve, outputs matrix
def laplacianMatrix(c, identityCoeff=1):
    N = len(c)
    L = np.zeros((N, N), dtype=np.float64)

    edgeLengths = [c.edgeLength(i) for i in range(N)]
    # Fill row
    for i in range(N):
        fi = 0.5 * np.linalg.norm(c[i+1] - c[i-1])
        denominator = edgeLengths[i-1] * edgeLengths[i] * fi
        L[i][i-1] = edgeLengths[i] / denominator
        L[i][i] = - (edgeLengths[i-1] + edgeLengths[i]) / denominator
        L[i][(i+1) % N] = edgeLengths[i-1] / denominator
    
    return L + identityCoeff * np.eye(N)

figure8 = figure8_generate()
alpha = 2
beta = 4
deltaT = 0.004
identityCoeff = -0.1
plotFreq = 100
H1Energies = []

# Plot
for t in range(M):
    # Compute derivative
    dE = quadrature4.dEnergy(figure8, alpha, beta).list_of_points
    # Solve Linear System
    L = laplacianMatrix(figure8, identityCoeff=identityCoeff)
    G = curve.Curve(np.linalg.solve(L, dE))
    # Evolve
    figure8 = G * deltaT + figure8
    H1Energies.append(quadrature4.energy(figure8.list_of_points))
    print(f"Progress: {t} / {M}")

In [ ]:
# Save Progress
with open("H1Energies.json", "w") as file:
    json.dump(H1Energies, file)
    file.close()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.plot(L2Energies, color="orange")
ax.plot(H1Energies, color="blue")
plt.show()

In [ ]:
def curveFromFourierCoefficients(xa, xb, ya, yb, za, zb, resolution=90):
    J = len(xa) - 1

    def fourierFromCoefficient(an, bn):
        return lambda x : an[0]/2 + sum([np.cos(i*x)*an[i] + np.sin(i*x)*bn[i] for i in range(1, J + 1)])
    
    fx = fourierFromCoefficient(xa, xb)
    fy = fourierFromCoefficient(ya, yb)
    fz = fourierFromCoefficient(za, zb)

    points = []
    for i in range(resolution):
        theta = 2 * np.pi * i / resolution
        points.append(np.array([
            fx(theta),
            fy(theta),
            fz(theta)
        ]))
    
    return curve.Curve(points)


def curveFourierDifferential(xa, xb, ya, yb, za, zb, perturbation=0.001, resolution=90, energyFunction=quadrature4.energy):
    J = len(xa) - 1

    variables = np.concatenate([xa, xb, ya, yb, za, zb])
    differential = np.zeros(len(variables), dtype="float64")
    for i in range(len(variables)):
        temp = variables[i]
        variables[i] += perturbation
        xac, xbc, yac, ybc, zac, zbc = np.split(variables, 6)
        tempCurve = curveFromFourierCoefficients(xac, xbc, yac, ybc, zac, zbc, resolution=resolution)
        energyP = energyFunction(tempCurve.list_of_points)
        variables[i] = temp
        variables[i] -= perturbation
        xac, xbc, yac, ybc, zac, zbc = np.split(variables, 6)
        tempCurve = curveFromFourierCoefficients(xac, xbc, yac, ybc, zac, zbc, resolution=resolution)
        energyN = energyFunction(tempCurve.list_of_points)
        differential[i] = (energyP - energyN) / (2 * perturbation)
        variables[i] = temp
    
    return differential

In [ ]:
fourierEnergies = []
xa = np.array([0.0, 1.0, 0.0])
xb = np.array([0.0, 0.0, 0.0])
ya = np.array([0.0, 0.0, 0.0])
yb = np.array([0.0, 0.0, 1.0])
za = np.array([0.0, 0.0, 0.0])
zb = np.array([0.0, 0.2, 0.0])
resolution = 30
energyFunction = quadrature4.energy
stepsize = 0.004

x = np.concatenate([xa, xb, ya, yb, za, zb])
xpoints = []
ypoints = []
zpoints = []
for t in range(M):
    xac, xbc, yac, ybc, zac, zbc = np.split(x, 6)
    differential = curveFourierDifferential(
        xac, xbc, yac, ybc, zac, zbc, perturbation=0.0001, resolution=resolution, energyFunction=energyFunction)
    x = x - stepsize * differential

    tempcurve = curveFromFourierCoefficients(
        xac, xbc, yac, ybc, zac, zbc, resolution=resolution)
    
    fourierEnergies.append(quadrature4.energy(tempcurve.list_of_points))
    
    print(f"Progress: {t} / {M}")

In [ ]:
with open("FourierEnergies.json","w") as file:
    json.dump(fourierEnergies, file)
    file.close()